In [8]:
SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose.S",    
    "Adipose-Visceral":"Adipose.V",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "Mucosa",
    "Esophagus-Muscularis":"Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve.T",
    "Skin-NotSunExposed": "SkinUnexposed",
    "Skin-SunExposed":"SkinLeg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood",
    "permuted":"Permuted"
#    "LCL": "LCL"
}
tissu = sorted([item for item in list(SHORTEN.keys()) if item != "permuted"])
##
##This code summarises the Anova results and prepare file for anovapvalue qqplot 
##

"""
delta_bic = snp_bic - snpstr_bic        ##delta_aic = snp_aic - snpstr_aic 
the lower tht aic or bic the closer the data is to the model
"""


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as ss

for Tissue in tissu:
    
    print('\nSummary for '+Tissue+' ...')

    #FDR 
    pval = pd.read_csv("/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"+Tissue+"/PQValues", sep='\t')
    pval = pval.loc[pval['qvalue']<=0.1]  ##eSTRs at 10% FDR
    print('There are ',pval.shape[0],' eSTRs at 10% FDR')

    #Anova output 
    hh = pd.read_csv("/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"+Tissue+"/HH/anova_wg.csv", sep=',')
    x=hh.loc[(hh['delta_aic']>0) & (hh['delta_bic']>0)].shape[0]
    print( 'ANOVA: ',x,' out of ',hh.shape[0], 'favor SNP+STR model over all ie ', x*100/hh.shape[0] ,'%')
    hh1 = hh.loc[hh['gene'].isin(pval['gene'])] 
    print('\t',hh1.shape[0], ' eSTRs with anova p-values')
    
    hh2 = hh1.loc[(hh1['delta_aic']>0) & (hh1['delta_bic']>0)]
    print('\t',hh2.shape[0], ' eSTRs addition to SNP model fit the data better than SNP only')
    print('\t ', hh2.loc[hh2['anova_pval']<=0.05].shape[0], ' have significant pvalues on anova test >=0.05')
    

    #hh1.to_csv("/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"+Tissue+"/HH/ESTRs_Anova", sep='\t', index=False)

## run 


Summary for Adipose-Subcutaneous ...
There are  1353  eSTRs at 10% FDR
ANOVA:  573  out of  11945 favor SNP+STR model over all ie  4.796986186688991 %
	 1310  eSTRs with anova p-values
	 207  eSTRs addition to SNP model fit the data better than SNP only
	  206  have significant pvalues on anova test >=0.05

Summary for Adipose-Visceral(Omentum) ...
There are  693  eSTRs at 10% FDR
ANOVA:  560  out of  11819 favor SNP+STR model over all ie  4.738133513833658 %
	 660  eSTRs with anova p-values
	 111  eSTRs addition to SNP model fit the data better than SNP only
	  111  have significant pvalues on anova test >=0.05

Summary for Artery-Aorta ...
There are  821  eSTRs at 10% FDR
ANOVA:  589  out of  11197 favor SNP+STR model over all ie  5.260337590426007 %
	 766  eSTRs with anova p-values
	 117  eSTRs addition to SNP model fit the data better than SNP only
	  117  have significant pvalues on anova test >=0.05

Summary for Artery-Tibial ...
There are  1257  eSTRs at 10% FDR
ANOVA:  562  ou

In [ ]:
Summary for WholeBlood ...

630  eSTRs at 10% FDR
611  eSTRs and their anova p-values

Summary for Cells-Transformedfibroblasts ...

955  eSTRs at 10% FDR
916  eSTRs and their anova p-values

Summary for Muscle-Skeletal ...

304  eSTRs at 10% FDR
292  eSTRs and their anova p-values

Summary for Lung ...

378  eSTRs at 10% FDR
378  eSTRs and their anova p-values

Summary for Adipose-Subcutaneous ...

376  eSTRs at 10% FDR
360  eSTRs and their anova p-values

Summary for Artery-Tibial ...

524  eSTRs at 10% FDR
504  eSTRs and their anova p-values

Summary for Esophagus-Mucosa ...

475  eSTRs at 10% FDR
460  eSTRs and their anova p-values

In [1]:
##This code summarizes ANOVA output and print out significant anova pval  with estrs
import pandas as pd
import numpy as np

Tissue="WholeBlood"
INN='/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Cells-Transformedfibroblasts/'
### Qval results STRs

print('STRs...')
#Get most signif. STR by gene from linear reg STRs
LR1=pd.read_csv(INN+"/Lin_Reg_OutFin.txt", '\t',low_memory=False)
idx = LR1.groupby(['gene'])['p.wald'].transform(min) == LR1['p.wald']
LR = LR1[idx]

#FDR corrected STRs
pq=pd.read_csv(INN+"/PQValue.tsv", sep='\t')
print(LR1.shape, '  to  ', LR.shape, ' PQ ',pq.shape)   

### QVal results SNPs
#print('SNPs...')
### ANOVA results SNPs+STRs

print('ANOVA...')
#Anova output 
ANO = pd.read_csv(INN+"/HH/anova_wg.csv",sep=',' )
print(ANO.shape) 

### checking eSTRs (FDR 5%)
estr = pq.loc[pq['qvalue']<=0.05]
estr.to_csv(INN+'HH/estr_regression.csv', sep='\t')
#perm=permuted.loc[permuted['str.id'].isin(list(estr['str.id']))]
print(estr.shape[0],' eSTRs at 5% FDR')

# eSTRs
ANsig = ANO.loc[ANO['anova_pval']<=0.05]                    #anova significant
print(ANsig.shape[0], ' Anova tests with pvalues<=0.05')
ANsig = ANsig.loc[ANsig['gene'].isin(list(estr['gene']))] #significant anova with estrs
#snp1=LR0.loc[LR0['gene'].isin(list(AN1['gene']))]  # cis snp for estrs

Anova_estr=ANO.loc[ANO['gene'].isin(list(estr['gene']))]
Anova_estr.to_csv(INN+'HH/Anova_estr.csv', sep='\t')
print(Anova_estr.shape)

#delta_aic = aic_snp-aic_snpstr
aicsnpstr = ANO.loc[ANO['delta_aic']>0]
bicsnpstr = ANO.loc[ANO['delta_bic']>0]
ea_snpstr = ANsig.loc[ANsig['delta_aic']>0]
eb_snpstr = ANsig.loc[ANsig['delta_bic']>0]
print('\nBy AIC',aicsnpstr.shape[0],' anova tests that favored the SNP+STR model  &  ' ,ea_snpstr.shape[0],' Significant anova tests that favored the SNP+STR model')
print('By BIC',bicsnpstr.shape[0],' anova tests that favored the SNP+STR model  &  ' ,eb_snpstr.shape[0],' Significant anova tests that favored the SNP+STR model')
print(ANsig.shape[0], ' significant anova with estrs\n')

# Expression
ANsig = ANsig.sort(['anova_pval'])
#ANsig.loc[ANsig['chrom']=='chr20']
ANsig

STRs...
(217114, 13)   to   (14806, 13)  PQ  (14804, 11)
ANOVA...
(14804, 13)
666  eSTRs at 5% FDR
3670  Anova tests with pvalues<=0.05
(666, 13)

By AIC 7849  anova tests that favored the SNP+STR model  &   169  Significant anova tests that favored the SNP+STR model
By BIC 2647  anova tests that favored the SNP+STR model  &   139  Significant anova tests that favored the SNP+STR model
169  significant anova with estrs



/usr/lib/python3.4/site-packages/ipykernel/__main__.py:54: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


chrom                gene  str.start  numsnps  numsamples    r2_str  \
6040    chr7   ENSG00000243566.2   76588651        1         117  0.636388   
7796   chr10   ENSG00000151640.8  134000824        1         119  0.552272   
8863   chr11   ENSG00000110455.9   44119312        1         113  0.167440   
852     chr1  ENSG00000122497.13  148009710        1         115  0.142084   
11329  chr15   ENSG00000184206.7   84972753        1         120  0.223902   
14370  chr21   ENSG00000243440.2   15938851        1         117  0.153997   
3860    chr4   ENSG00000109163.6   68621932        1         121  0.122404   
11391  chr15   ENSG00000188659.5   82491654        1         121  0.358625   
8448   chr11   ENSG00000170962.8  103903778        1          88  0.165590   
12552  chr17  ENSG00000141485.11    6690543        1         119  0.144235   
13356  chr18   ENSG00000197563.5   59817930        1         120  0.120621   
11665  chr16  ENSG00000187555.10    9115017        1          85  0.172974   
8290   chr10  ENSG00000172671.15   46090048        1          98  0.267040   
1002    chr1  ENSG00000143434.11  151081775        1          79  0.206807   
1284    chr1   ENSG00000204060.4   41728517        1         124  0.150933   
8288   chr10  ENSG00000095585.12   98106815        1         119  0.139700   
14427  chr21   ENSG00000160213.5   45196326        1         122  0.255945   
4997    chr6   ENSG00000124587.9   42926477        1         125  0.430170   
10017  chr12   ENSG00000111666.6  102087516        1         122  0.178363   
3293    chr3   ENSG00000168038.6   41905839        1         123  0.374106   
13415  chr19   ENSG00000167657.7    4026122        1          73  0.202872   
1850    chr2  ENSG00000115718.13  128256658        1         102  0.131624   
698     chr1   ENSG00000196420.3  153559700        1         121  0.103053   
1966    chr2   ENSG00000119878.5   46890408        1         119  0.113371   
13671  chr19   ENSG00000197013.5   21761715        1         124  0.135212   
9322   chr12   ENSG00000010278.7    6255391        1         125  0.112916   
9444   chr12   ENSG00000139182.9    7246582        1         125  0.113791   
14479  chr22   ENSG00000133433.6   24292861        1         117  0.315296   
14380  chr21   ENSG00000182362.9   47678650        1         119  0.216443   
8175   chr10   ENSG00000188234.9   46292054        1         116  0.172367   
...      ...                 ...        ...      ...         ...       ...   
4477    chr5   ENSG00000150712.6   32317804        1         110  0.149645   
11797  chr16  ENSG00000076344.11     220831        1         116  0.115787   
12627  chr17   ENSG00000171298.8   78101441        1         121  0.147720   
2413    chr2  ENSG00000180902.12  242686808        1         125  0.278782   
2952    chr3   ENSG00000188958.5  191082878        1         121  0.147273   
5172    chr6   ENSG00000118514.9  135339115        1         108  0.231374   
2557    chr2   ENSG00000115816.9   37462783        1         124  0.204814   
2825    chr3  ENSG00000178053.13  158302448        1         113  0.125418   
13809  chr19   ENSG00000105072.4   16593190        1         117  0.160751   
686     chr1  ENSG00000142794.14   21783661        1         116  0.197961   
11731  chr16   ENSG00000065457.6   75614791        1         122  0.133883   
8461   chr11   ENSG00000176485.6   63382865        1         109  0.246824   
10274  chr13   ENSG00000120675.4   43656981        1         118  0.308352   
11227  chr15  ENSG00000080644.11   78961264        1         112  0.153688   
9514   chr12  ENSG00000170473.12   56326477        1         120  0.130104   
4539    chr5   ENSG00000164338.5   72898709        1         114  0.127277   
10967  chr15   ENSG00000182117.4   34602376        1         105  0.194929   
9162   chr11   ENSG00000175575.8   73637141        1         114  0.285400   
91      chr1   ENSG00000181754.6  110052993        1         122  0.277780   
12079  chr16   ENSG0000012992

In [2]:
#Find lead snp for chosen use case to plot
#Genes=['ENSG00000181754.6', 'ENSG00000152760.5','ENSG00000242485.1']
Genes=list(ANsig['gene'])
SNP = pd.read_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+Tissue+"/SNP_Analysis/PQValues.txt", sep='\t')
SNP = SNP.loc[SNP['gene'].isin(Genes)]
SNP = SNP.sort_values(by=["chrom", "str.start"], ascending=[True, True])
SNP = SNP.loc[:,['gene','chrom','str.start','p.wald','qvalue','beta']]
print (SNP.loc[SNP['chrom']=='chr4'])

                     gene chrom    str.start    p.wald    qvalue      beta
13185   ENSG00000189308.6  chr4   83941387.0  0.001452  1.000000 -0.263854
7486   ENSG00000145476.11  chr4  187166893.0  0.000031  0.289261 -0.340693


In [3]:
 python ~/projects/GTEX_eSTRs/gtex-estrs/Scripts/Heritability/AnalyzeHiddenHeritability.py --estrs estr_result --esnps esnp_result --expr ../Corr_Expr.csv
--exprannot /storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv  --chrom $ch --strgt /storage/szfeupe/Runs/650GTEx_estr/Genotypes/STR_Norm_lized_Geno.table  --snpgt /storage/szfeupe/Runs/650GTEx_estr/SNP_Analysis/chr$1.tab --str-fdr-threshold 1 > anova_ch$ch


strgenos='/storage/szfeupe/Runs/650GTEx_estr/SNP_Analysis/chr'$1.tab
EXPRAN='/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'

chrom                gene  str.start  numsnps  numsamples    r2_str  \
3860  chr4   ENSG00000109163.6   68621932        1         121  0.122404   
3643  chr4   ENSG00000189308.6   83938708        1         116  0.156043   
4194  chr4  ENSG00000145476.11  187132811        1         114  0.136842   

        r2_snp  r2_snpstr  anova_pval  estr_fdr  esnp_fdr  delta_bic  \
3860  0.113063   0.237204    0.000026  0.029879  0.450593  13.448960   
3643  0.094647   0.156697    0.004710  0.009803  1.000000   3.482256   
4194  0.257940   0.285306    0.041578  0.028726  0.002325  -0.452474   

      delta_aic  
3860  16.244751  
3643   6.235846  
4194   2.283724